# habspec

> Load, Calibrate, HABSpec json spectra files captured by the HABSpec.

In [ ]:
#| default_exp habspec

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev

In [ ]:
#| hide
nbdev.nbdev_export()

In [ ]:
#| export
import glob
import numpy             as np
import pandas            as pd
import cv2               as cv
import datetime          as dt
import glob
import matplotlib.pylab  as plt
import numpy             as np
import pandas            as pd
import pandas            as pd
import time
from geographiclib.geodesic import Geodesic
from           PIL          import Image   
from       skimage          import io

In [ ]:
import RedTideProcessing.wavelength_cal     as cal
import RedTideProcessing.testing            as testing
import RedTideProcessing.habspec_data_class as hsdc

In [ ]:
## from google.colab.patches   import cv2_imshow # for image display

#| hide
## Global vars for testing only.

For working with test data from s:

Gps_File_Name = '/mnt/f/My Drive/Missions/2021-1208-HAB-Lake-Parker/'\
                '2021-1208/2021-1208-175324-910850-mission-gps.txt'

mission_folder = '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/'

Gps_File_Name = "/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/2021-0717-044645-687230-mission-gps.txt"

test_spectra_file = mission_folder+'/160206/hab_spectra/2021-0717-160401-066388-spec.json'

mission_folder = "../sample_data/2023-0718/034901/"
test_spectra_file = mission_folder + "/hab_spectra/" + "2023-0718-034912-259843-spec.json"
test_spectra_file

#| hide
## Module wide Globals

#| hide
### Working spectra data 

#| export
class HAB_Spectra:
  '''
  '''
  Xpixels     = np.array(1)
  wavelengths = np.array(1)
  create_time = dt.datetime.utcnow()
  pixel0      = None
  wavelength0 = None
  pixel1      = None
  wavelength1 = None
  def __init__(self):
    self.json_spectra_file_name    = None
    self.Lat          = None
    self.Lon          = None
    self.altitde_m    = None
    self.n_saturated  = None
    self.raw_y_min    = None
    self.remove_bias  = None
    self.y_average    = None
    
    self.DateTime     = None
    self.Exposure     = None
    
    self.raw_y        = []
    self.summed_rows  = 0
    
  def __str__(self):
    rv = ''
    for k in self.__dict__.keys():
      rv += f'{k:20s}: {self.__dict__[k]}\n'
    return rv
    

In [ ]:
hab_spectra = hsdc.HAB_Spectra()

In [ ]:
print(hab_spectra.__str__())

json_spectra_file_name: None
Lat                 : None
Lon                 : None
altitde_m           : None
n_saturated         : None
raw_y_min           : None
remove_bias         : None
y_average           : None
DateTime            : None
Exposure            : None
raw_y               : []
summed_rows         : 0



#| hide
## Functions

#| hide#
### General Purpose datetime Functions

#| hide
#### def hhmmss_to_sod( hhmmss, Usecs=0):

In [ ]:
#| export
def hhmmss_to_sod( hhmmss, Usecs=0):
    """
    Converts a time string in 'HHMMSS' format to a seconds-of-the-day
    including an optional microseconds fraction.

    Parameters:
    ----------- 
    hhmmss : str
    Usecs  : str, default=0

    Returns: 
    --------
    float
      Seconds of the day including the fractional part computed from `Usecs`
    
    Description:
    ------------
    hhmmss where hh is hours, mm is minutes, and ss is seconds. Example '123456'
    is 12 hours, 34 minutes, and 56 seconds. A string representing the number of
    microseconds since the last second increment, ie the fractional part of a second. 
    Example: '954561' represents 0.954561 seconds, or 954,561 microseconds since the
    last seconds rollover. Returns A single floating point value of the seconds since
    midnight plus the fractional seconds.

    Examples:
    ---------
    To be added.

    """
    hh = int(hhmmss[0:2])
    mm = int(hhmmss[2:4])
    ss = int(hhmmss[4:6])
    fsod = hh*3600 + mm*60 + ss + float(Usecs) * 1e-6
    return fsod

#| hide
#### def now_utc( fmt='%Y-%m%d %H:%M:%S'):

In [ ]:
#| export
def now_utc( fmt='%Y-%m%d %H:%M:%S'):
    """ Return the current UTC date and time as a string.  Example return: '2021-0812 15:39:41'.
    See: http://shorturl.at/koOQ7 for timeformat options & directives.
    """
    t = dt.datetime.utcnow()
    ts = f'{t:{fmt}}'
    return ts

In [ ]:
now_utc()

'2023-0724 21:03:53'

#| hide
#### def hms2sod( str, debug=False):

In [ ]:
#| export
def hms2sod( str ):
  '''
  Converts an ASCII string in the form 'HH:MM"SS' to seconds of the day. Works with
  fractional seconds.
  '''
  t_lst = str.split('.')
  ts = time.strptime( t_lst[0], '%H:%M:%S')
  if len(t_lst) > 1:
    fs = float("."+t_lst[1] )
  else:
    fs = 0.0
  sod = ts[3]*3600 + ts[4]*60 + ts[5] + fs
  return sod

In [ ]:
print('\nTesting: hms2sod( t )')
tt = ['00:00:00',   '23:59:59', '12:00:00',
      '00:00:00.0', '0:0:0.0', '0:0:0' ]
print('Time Str       Seconds of Day\n'\
       '-----------------------------')
for t in tt:
  rv = hms2sod( t )
  print(f'{t:16} {rv:12.6f}')


Testing: hms2sod( t )
Time Str       Seconds of Day
-----------------------------
00:00:00             0.000000
23:59:59         86399.000000
12:00:00         43200.000000
00:00:00.0           0.000000
0:0:0.0              0.000000
0:0:0                0.000000


|! hide
####   def extract_hms( fn):

In [ ]:
#| export
def extract_hms( 
  fn:str         # Filename 
) ->tuple:       # ( hhmmss:str, fsecs:str, hhmmss.fsecs:float, sod.fsecs:float )
    '''Extract the HHMMSS and Fsecs strings from filename "fn"

    Returns:
    --------
    ( hhmmss:str, fsecs:str, hhmmss.fsecs:float, sod.fsecs:float )
    '''
    lst = fn.split('-')
    sod = hhmmss_to_sod( lst[-3], Usecs=lst[-2] )
    return ( lst[-3], lst[-2], float(lst[-3])+float(lst[-2])*1e-6, sod  )

#| hide
### HABSpec JSON Functions

#| hide
#### def configure_json_spectra()

In [ ]:
#| export
def configure_json_spectra(
  hab_spectra_class,  # The HAB_Spectra class to update.
  file_name:str       # Json settings file.
) ->bool:              # Returns True if successful, else False.
    '''
    Configures HabSpec internal setting according to settings found within a json spectra file 'file_name'.
    '''
    try:
      spec = pd.read_json(file_name)                                                     # get a spectra fron a json file to pandas.
    except:
      hab_spectra_class.error_msgs = f"configure_json_spectra() failed. Unable to read: {file_name}."
    try:
      hab_spectra_class.json_spectra_file_name = file_name
      hab_spectra_class.raw_y                  = y = np.array(spec['hab_spec'].spectra ) # Load the y values to an np array
      hab_spectra_class.summed_rows            = spec['hab_spec'].summed_rows
      hab_spectra_class.Xpixels                = np.arange( 0 ,y.size )                  # Construct an array of X values from 0 to y.size
      hab_spectra_class.wavelengths            = np.arange(    y.size )                  # construce, but don't fillin. Leave for calibration.
      rv = True
      hab_spectra_class.error_msgs = ""
      hab_spectra_class.Configured = True
    except:
      rv = False
      hab_spectra_class.error_msgs = "configure_json_spectra() failed trying to set variables."
    return rv

In [ ]:
fn = '../sample_data/2023-0718/034901/hab_spectra/2023-0718-034901-714246-spec.json'
rv = configure_json_spectra(hab_spectra,  fn )
rv, hab_spectra.Xpixels, hab_spectra.summed_rows, hab_spectra.wavelengths

(True,
 array([   0,    1,    2, ..., 1277, 1278, 1279]),
 800,
 array([   0,    1,    2, ..., 1277, 1278, 1279]))

In [ ]:
print(hab_spectra.__str__())

json_spectra_file_name: ../sample_data/2023-0718/034901/hab_spectra/2023-0718-034901-714246-spec.json
Lat                 : 28.209754667
Lon                 : -82.3639295
altitde_m           : None
n_saturated         : None
raw_y_min           : 36776
remove_bias         : True
y_average           : False
DateTime            : 2023-0718-034909.929
Exposure            : 500.0
raw_y               : [36800 36800 36800 ... 36896 36896 36896]
summed_rows         : 800
Xpixels             : [   0    1    2 ... 1277 1278 1279]
wavelengths         : [   0    1    2 ... 1277 1278 1279]
pixel0              : 73
pixel1              : 941
wavelength0         : 430.774
wavelength1         : 759.37
delta_pix           : 868
delta_wavelength_nm : 328.596
Calibrated_w        : True
GPS_Alt             : 22.3
GPS_Nsat_Inview     : 6
GPS_Status          : 3
GPS_UTC             : 03:49:08
Source              : Water
Pix_minv            : 36776
sat_pix_n           : 0
Pix_maxv            : 44195
error_ms

|! hide
#### def calibrate_using_2_wavelengths()

In [ ]:
#| export
def calibrate_using_2_wavelengths(
                                  hab_spectra_class:object, # The HAB_Spectra class to update.
                                  spectra_file_name:str,    # Filename of a json spectra file.
                                  pixel0:int=0,             #  pixel # for Wavelength 0
                                  wavelength0:float=0,      # Wavelength of pixel 0
                                  pixel1:int=0,             # pixel # for Wavelength 1
                                  wavelength1:float=0       # Wavelength for pixel1
                                 ) ->bool:                  # True on success, False otherwise.
    '''
    Generate a Numpy array of calibration wavelenghts for each pixel. configure_json_spectra(f)
    must be called beforehand inrder to set the correct number of pixels.

    Example:
    ---------
    # Configure the spectra by reading a spectra file.
    fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
    hs.configure_json_spectra(fn)

    # Calibrate the spectra.  Pixel 73 is at 430.774nm, and pixel 941 is at 759.370nm
    hs.calibrate_using_2_wavelengths(pixel0=73, wavelength0=430.774, pixel1=941, wavelength1=759.370 )

    References:
    -----------
    The following are wavelength calibration sources.
    See: http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/atspect2.html
    and: https://commons.wikimedia.org/wiki/File:Fluorescent_lighting_spectrum_peaks_labelled.gif
    and: https://en.wikipedia.org/wiki/Fluorescent_lamp#/media/File:Spectra-Philips_32T8_natural_sunshine_fluorescent_light.svg
    '''
    rv = configure_json_spectra(hab_spectra_class,  spectra_file_name )
    if rv:
      try:
        hab_spectra_class.pixel0              = pixel0
        hab_spectra_class.pixel1              = pixel1
        hab_spectra_class.wavelength0         = wavelength0
        hab_spectra_class.wavelength1         = wavelength1

        dp                                    = pixel1 - pixel0            # pixel delta
        hab_spectra_class.delta_pix           = dp 

        dw                                    = wavelength1 - wavelength0  # Wavelength delta
        hab_spectra_class.delta_wavelength_nm = dw

        slope                                 = dw/dp                      # Linear slope
        hab_spectra_class.wavelengths         = wavelength0 + (hab_spectra_class.Xpixels-pixel0) * slope
        hab_spectra_class.Calibrated_w        = True
        rv = True
      except:
        rv = False
        hab_spectra_class.error_msgs += " calibrate_using_2_wavelengths() failed."
    return rv

Calibrate spectra by setting the pixel numbers and cooresponding wavelengths
for two pixels.  The two pixels should be a far apart as possible to minimize
error.  The calibrate_using_2_wavelengths() function will return a list 
which contains the wavelength for every pixel in the spectra.

In [ ]:
# Calibrate the spectra for wavelength in nanometers
rv = calibrate_using_2_wavelengths( 
  hab_spectra,
  fn,
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 )

if rv:
  print(f'Configured using: {hab_spectra.json_spectra_file_name}')
  print('\nWavelength Calibration Data:\n\n'
  '                     First      Second\n'
  f'   Wavelengths (nm): {hab_spectra.wavelengths[73]:7.3f}      {hab_spectra.wavelengths[941]:7.3f}\n'
  f'     Pixel Location: {73:7.3f}      {941:7.3f}\n')

  print(f'{len( hab_spectra.wavelengths )} elements found in `wavelengths`.' )
  print(f'First 5 pixel wavelengths: {hab_spectra.wavelengths[0:5]}')
  print(f' Last 5 pixel wavelengths: {hab_spectra.wavelengths[-5:]}')
else:
  print('calibrate_using_2_wavelengths() returned False.  It failed')

Configured using: ../sample_data/2023-0718/034901/hab_spectra/2023-0718-034901-714246-spec.json

Wavelength Calibration Data:

                     First      Second
   Wavelengths (nm): 430.774      759.370
     Pixel Location:  73.000      941.000

1280 elements found in `wavelengths`.
First 5 pixel wavelengths: [403.13862212 403.51718894 403.89575576 404.27432258 404.6528894 ]
 Last 5 pixel wavelengths: [885.81131797 886.18988479 886.56845161 886.94701843 887.32558525]


|! hide
#### def read_spectra()

In [ ]:
#| export
def read_spectra( hab_spectra_class,      # The HAB_Spectra class to update.                
                 spectra_json_file_name,  # JSON Spectra file name
                   remove_bias = True,    # Bol remove bias
                   y_average   = True     # Average Y values
                  ) -> float:             # Numpy array of spectral points.
    """
    Reads a Json hyperspectra file.  Note: Any data older than 2022-12 does not have
    GPS info in the json spectra.  The GPS data is recorded in a separate file.

    Parameters:
    -----------
    spectra_json_file_name : str
      The full path name of a hyperspectra Json file.
    remove_bias : Default True
      Subtracts the minimum y value from the array of y values to remove dark current. 
      The minimum value will typically be found in the IR side and from the pixel 
      sensors that are optically obscured.
    y_average : Default True
      The Json values are the sum total of each pixel column. Each column contains 800
      pixels. Setting this to True causes the read y values to be divided by 800.

    Returns:
    --------
    Numpy array
      An array of numpy float values representing the intensity values at each pixel. 

    Description:
    ------------
    Reads a Json spectra from a file.

    Examples:
    ---------
    fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
    s = hs.read_spectra(fn) 
    """
    spec = pd.read_json(spectra_json_file_name)
    hab_spectra_class.json_spectra_file_name = spectra_json_file_name
    try:
      hab_spectra_class.DateTime        = spec['hab_spec']['DateTime']
      hab_spectra_class.Exposure        = spec['hab_spec']['Exposure']
      hab_spectra_class.GPS_Alt         = spec['hab_spec']['GPS_Alt']
      hab_spectra_class.GPS_Nsat_Inview = spec['hab_spec']['GPS_Nsat_Inview']
      hab_spectra_class.GPS_Status      = spec['hab_spec']['GPS_Status']
      hab_spectra_class.GPS_UTC         = spec['hab_spec']['GPS_UTC']
      hab_spectra_class.Lat             = spec['hab_spec']['Lat']
      hab_spectra_class.Lon             = spec['hab_spec']['Lon']
      hab_spectra_class.Source          = spec['hab_spec']['Source']
      hab_spectra_class.Pix_minv        = spec['hab_spec']['Pix_minv']
      hab_spectra_class.sat_pix_n       = spec['hab_spec']['sat_pix_n']
      hab_spectra_class.Pix_maxv        = spec['hab_spec']['Pix_maxv']
      hab_spectra_class.Pix_maxv        = spec['hab_spec']['Pix_maxv']
    except:
      hab_spectra_class.DateTime        = None
      hab_spectra_class.Exposure        = None
      hab_spectra_class.GPS_Alt         = None
      hab_spectra_class.GPS_Nsat_Inview = None
      hab_spectra_class.GPS_Status      = None
      hab_spectra_class.GPS_UTC         = None
      hab_spectra_class.Lat             = None
      hab_spectra_class.Lon             = None
      hab_spectra_class.Source          = None
      hab_spectra_class.Pix_minv        = None
      hab_spectra_class.sat_pix_n       = None
      hab_spectra_class.Pix_maxv        = None
      hab_spectra_class.Pix_maxv        = None
    try:
      hab_spectra_class.summed_rows     = spec['hab_spec']['summed_rows']
    except:
      pass
      
      
    hab_spectra_class.json_spectra_file_name       = spectra_json_file_name
    hab_spectra_class.remove_bias     = remove_bias
    hab_spectra_class.y_average       = y_average
    
    if hab_spectra_class.Configured == 0 :
      configure_json_spectra(hab_spectra_class, spectra_json_file_name)
      
    hab_spectra_class.raw_y = np.array(spec['hab_spec'].spectra)
    
    hab_spectra_class.raw_y_min = hab_spectra_class.raw_y.min()
    
    if remove_bias == True:
      hab_spectra_class.raw_y = hab_spectra_class.raw_y - hab_spectra_class.raw_y_min

    if y_average == True:
      hab_spectra_class.raw_y = hab_spectra_class.raw_y / hab_spectra_class.summed_rows

    return hab_spectra_class.raw_y

In [ ]:
test_json_file0 = "../sample_data/2023-0718/034901/hab_spectra/2023-0718-034909-929994-spec.json"
spec_test = pd.read_json(test_json_file0)
display(test_json_file0, spec_test)

'../sample_data/2023-0718/034901/hab_spectra/2023-0718-034909-929994-spec.json'

,hab_spec,Event_t
DateTime,2023-0718-034909.929,1.689652e+09
Exposure,500.0,1.689652e+09
GPS_Alt,22.3,1.689652e+09
GPS_Nsat_Inview,6,1.689652e+09
GPS_Status,3,1.689652e+09
GPS_UTC,03:49:08,1.689652e+09
Lat,28.209755,1.689652e+09
Lon,-82.363929,1.689652e+09
Pix_maxv,44195,1.689652e+09
Pix_minv,36776,1.689652e+09


In [ ]:
print(test_json_file0)
test_spectra_y = read_spectra(hab_spectra, test_json_file0, y_average=False)
print(hab_spectra.__str__())
display('Raw spectra, every 25th value:',hab_spectra.raw_y[0:-1:25])

../sample_data/2023-0718/034901/hab_spectra/2023-0718-034909-929994-spec.json
json_spectra_file_name: ../sample_data/2023-0718/034901/hab_spectra/2023-0718-034909-929994-spec.json
Lat                 : 28.209754667
Lon                 : -82.3639295
altitde_m           : None
n_saturated         : None
raw_y_min           : 36776
remove_bias         : True
y_average           : False
DateTime            : 2023-0718-034909.929
Exposure            : 500.0
raw_y               : [ 24  24  24 ... 184 184 184]
summed_rows         : 800
Xpixels             : [   0    1    2 ... 1277 1278 1279]
wavelengths         : [403.13862212 403.51718894 403.89575576 ... 886.56845161 886.94701843
 887.32558525]
pixel0              : 73
pixel1              : 941
wavelength0         : 430.774
wavelength1         : 759.37
delta_pix           : 868
delta_wavelength_nm : 328.596
Calibrated_w        : True
GPS_Alt             : 22.3
GPS_Nsat_Inview     : 6
GPS_Status          : 3
GPS_UTC             : 03:49:08
S

'Raw spectra, every 25th value:'

array([  24,   24,   24,  264, 1680, 5528, 5900, 3664, 2312, 2416, 3040,
       3884, 5096, 5600, 5737, 5915, 6199, 6361, 6528, 6900, 7216, 7389,
       6964, 6269, 5220, 4274, 3328, 2480, 1856, 1096,  664,  456,  280,
         72,   24,   24,   24,   24,   24,   24,   24,   40,   24,   24,
         24,   24,   24,   24,   24,   24,   40,  184])

In [ ]:
import sys
sys.getsizeof( hab_spectra )

56

#| hide
####   def get_list_of_json_spectra( p):

In [ ]:
#| export
def get_list_of_json_spectra( p        # String path to mission data set.
                            ) ->list:  # List of full paths to each JSON spectra file.
    ''' Returns a list of Json spectra full path filenames found in subdirs under "p". '''
    gstr = f'{p}/*/*/*-spec.json'
    lst = glob.glob(gstr, recursive=True)
    return lst

Test and demo get_list_of_json_spectra()

In [ ]:
mission_path = '../sample_data/2023-0718'
mission_path = '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/'
test_spectra_list = get_list_of_json_spectra( mission_path )
print(f'{len(test_spectra_list):,} JSON spectra files found.')
display("First 3", test_spectra_list[0:3])
display("Last 3",test_spectra_list[-3:])

11,273 JSON spectra files found.


'First 3'

['/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141626-175824-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141626-882736-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141627-601038-spec.json']

'Last 3'

['/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-170224-525748-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-170225-225927-spec.json',
 '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-170225-920240-spec.json']

#| hide
####   def json_specs_to_df( specs ):

In [ ]:
#| export
def json_specs_to_df( specs ):
    '''
    Extracts the hhmmss and fsecs from 'specs' filename, converts
    the hhmmss and fsecs strings to float SOD.fsecs.

    Parameters:
    specs: list  A list of json spectra files.

    Returns:
    --------
    A DataFrame of sod.fsecs:float, hhmmss:str, Json_spec:str
    '''
    # Extract the hhmmss and fsecs from each file name,
    # convert the hhmmss and fsecs to float SOD.fsecs, 
    # and create a list of each.
    hhmmss = []
    fsecs  = []
    sod    = []
    for v in specs:
      tx = extract_hms( v )
      hhmmss.append( tx[0] )   # Build the list if hhmmss strings.
      sod.append( tx[3] )      # Build the list of sod floats.

    # Now convert the  lists to a Pandas dataframe.
    dct = {'sod':sod, 'hhmmss':hhmmss, 'Json_spec':specs}
    df = pd.DataFrame( dct )
    df.sort_values(by=['sod'], inplace=True)              # Sort the data in time order.
    return df

Test configuring, calibrating, and loading a JSON spectra file.

In [ ]:
print(f'{dt.datetime.utcnow() }: Testing creating HyperSpectral.')

# Use a json spectra to initialize things.
print(f'{dt.datetime.utcnow() }: Loading a json Spectra file to calibrate wavelength.')

test_spectra_file = '../sample_data/2023-0718/034901/hab_spectra/2023-0718-034902-840971-spec.json'
# configure_json_spectra(hab_spectra, test_spectra_file)

# Calibrate the spectra for wavelength in nanometers
print(f'{dt.datetime.utcnow() }: Calibrating wavelength.')
wavelengths_test = calibrate_using_2_wavelengths(
  hab_spectra,
  test_spectra_file,
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 )

print('\nWavelength Calibration Data:\n\n'
'                     First      Second\n'
f'   Wavelengths (nm): {hab_spectra.wavelengths[73]:7.3f}      {hab_spectra.wavelengths[941]:7.3f}\n'
f'     Pixel Location: {73:7.3f}      {941:7.3f}\n')

# Read a spectra array from a file into spectra
print(f'{dt.datetime.utcnow() }: Reading a spectra into "spectra"')
spectra = read_spectra(hab_spectra, test_spectra_file )

# Plot the spectra vs pixel number.
#plt.plot(s)

2023-07-24 21:04:21.570034: Testing creating HyperSpectral.
2023-07-24 21:04:21.570170: Loading a json Spectra file to calibrate wavelength.
2023-07-24 21:04:21.570210: Calibrating wavelength.

Wavelength Calibration Data:

                     First      Second
   Wavelengths (nm): 430.774      759.370
     Pixel Location:  73.000      941.000

2023-07-24 21:04:21.573644: Reading a spectra into "spectra"


#### def is_water( x,y, debug=False):

In [ ]:
#| export
def is_water(
  hab_spectra_class  #
  #x,
  #y                 #
) -> float:          #
    '''
    Returns the mean signal value between 840nm and 860nm wavelengths.  Since water absorbs IR.

    Inputs:
      x   A numpy array of wavlengths for each pixel
      y   A numpy array of intensity values at each wavelength.  x and y mus be the same size.

    Returns:
      The mean signal value between 840nm and 860nm.  The signal level is not currently normalized
      for anything, exposure, etc.  Threshold is around 4.0.  Above 4, land or glint.

    References: 
    Application of the water-related spectral reflectance indices: A review
    https://www.sciencedirect.com/science/article/abs/pii/S1470160X18308215

    '''
    rv = hab_spectra_class.raw_y[hsdc.w_range_pixels( hab_spectra_class,840,860)].mean()
    return rv

In [ ]:
is_water( hab_spectra )

0.0

#| hide
#### def wavelength_2_RGB()

In [ ]:
#| export
def wavelength_2_RGB(
  wavelength,   # Wavelength in nanometers.
  alpha=255     # Alpha to use.
) -> list:      # ( red, green, blue )
  '''
  Convert a wavelength to a tuple of red, green, blue values.
  
  From: https://codingmess.blogspot.com/2009/05/conversion-of-wavelength-in-nanometers.html
  '''
  w = int(wavelength)

  # colour
  if w >= 380 and w < 440:
      R = -(w - 440.) / (440. - 350.)
      G = 0.0
      B = 1.0
  elif w >= 440 and w < 490:
      R = 0.0
      G = (w - 440.) / (490. - 440.)
      B = 1.0
  elif w >= 490 and w < 510:
      R = 0.0
      G = 1.0
      B = -(w - 510.) / (510. - 490.)
  elif w >= 510 and w < 580:
      R = (w - 510.) / (580. - 510.)
      G = 1.0
      B = 0.0
  elif w >= 580 and w < 645:
      R = 1.0
      G = -(w - 645.) / (645. - 580.)
      B = 0.0
  elif w >= 645 and w <= 780:
      R = 1.0
      G = 0.0
      B = 0.0
  else:
      R = 0.0
      G = 0.0
      B = 0.0

  # intensity correction
  if w >= 380 and w < 420:
      SSS = 0.3 + 0.7*(w - 350) / (420 - 350)
  elif w >= 420 and w <= 700:
      SSS = 1.0
  elif w > 700 and w <= 780:
      SSS = 0.3 + 0.7*(780 - w) / (780 - 700)
  else:
      SSS = 0.0
  SSS *= 255

  return [int(SSS*R), int(SSS*G), int(SSS*B), int(alpha)]

In [ ]:
w = 500
alpha = 255
rgb = wavelength_2_RGB( w, alpha=alpha )
print(f'{w=}    {rgb=}  {alpha=} ')

w=500    rgb=[0, 255, 127, 255]  alpha=255 


#| hide
#### def wavelength_2_RGB_str()

In [ ]:
#| export
def wavelength_2_RGB_str( 
  wavelength:float,        # Wavelength in nanometers.
  alpha=255                # Alpha, transparency to use.  
) ->str:                   # rgb string such as: "#01abffff"
  '''
  Converts a wavelength to an RGB string suitable to use
  as a color parameter in most applications.
  '''
  rgb  = wavelength_2_RGB(wavelength, alpha=alpha)
  rgbs = f'#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}{rgb[3]:02x}'
  return rgbs

In [ ]:
w     = 500
alpha = 127
rgbs  = wavelength_2_RGB_str( w, alpha=alpha )
print(f'{w=}    {rgbs=} ')

w=500    rgbs='#00ff7f7f' 


### Plot helpers

#| hide
#### def gen_reference_lines()

Generate spectral reference lines data suitable to plot with Bokeh.

#| export
def gen_reference_lines(
              hab_spec_class,   # Spectra class.
              #s,      # Spectra 
              lines,            # The reference lines.
             ) -> (list,list):  # Fraun X values, Fraun y values 
  '''
  Generate a series of spectral plot lines for reference.  It scales to the max y value 
  of the provided spectra within `hab_spec_class`.  Sources can be Fraunhofer_lines, or HG_lines, or user 
  developed.
  '''
  fran_x = []; fran_y = []
  y0 = 0.0;    y1 = max( hab_spec_class.raw_y)
  for i in lines:
    x = lines[i][1]
    fran_x.append( [ x,x ] )
    fran_y.append( [y0, y1] )
  return fran_x, fran_y

#| hide
# Testing.

#| hide
## Testing includes

In [ ]:
import panel            as pn
from bokeh.plotting import figure, show 
from bokeh.io       import show, output_notebook

In [ ]:
output_notebook()

Loading BokehJS ...

In [ ]:
spectra_number = 0
spectra_file, spectra_note = testing.select_test_file( spectra_number )

print(f'  Index: {spectra_number}')
print(f'   File: {spectra_file}')
print(f'   Note: {spectra_note}')

  Index: 0
   File: /mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json
   Note: No embedded GPS data. Gulf of Mexico west of Clearwater.


Calibrate the spectra for wavelength in nanometers

In [ ]:
test_wavelengths = calibrate_using_2_wavelengths( 
  hab_spectra,  spectra_file,
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 
)

### Generate a plot of the Spectra.

In [ ]:
spectra_y = read_spectra(hab_spectra,  spectra_file, y_average=True )

Configure the basics of the plot.

In [ ]:
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d
screen_height = 300
screen_width  = 700

In [ ]:
print(hab_spectra.__str__())

json_spectra_file_name: /mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json
Lat                 : None
Lon                 : None
altitde_m           : None
n_saturated         : None
raw_y_min           : 39253
remove_bias         : True
y_average           : True
DateTime            : None
Exposure            : None
raw_y               : [43.345   43.03625 43.3525  ...  0.30125  0.2475   0.26875]
summed_rows         : 800
Xpixels             : [   0    1    2 ... 1277 1278 1279]
wavelengths         : [403.13862212 403.51718894 403.89575576 ... 886.56845161 886.94701843
 887.32558525]
pixel0              : 73
pixel1              : 941
wavelength0         : 430.774
wavelength1         : 759.37
delta_pix           : 868
delta_wavelength_nm : 328.596
Calibrated_w        : True
GPS_Alt             : None
GPS_Nsat_Inview     : None
GPS_Status          : None
GPS_UTC             : None
Source              : None
Pix_minv            

In [ ]:
# Generate the Bokeh plot widget.
spectra_w = figure(       title = f"{ hab_spectra.json_spectra_file_name.split('/')[-1]},    {spectra_note}",
                   x_axis_label = 'Wavelength (nm)', 
                   y_axis_label = 'Intensity (Counts)',
                         height = 300, 
                          width= 700 
                  )

In [ ]:
# Generate Fraunhofer lines.
fran_x, fran_y = cal.gen_reference_lines( 
  cal.Fraunhofer_lines,
  y_max = max( hab_spectra.raw_y )
)

junk = spectra_w.multi_line(
  fran_x, 
  fran_y,
  color='lightgray',
  legend_label="Fraunhofer Lines."
)

# Plot Sentinel bands.
sentinel.sentinel_update( hab_spectra, sentinel_array )
for se in sentinel_array:
  spectra_w.rect(         x = sentinel_array[se].center_nm, 
                          y = sentinel_array[se].y_average, 
                      width = sentinel_array[se].width_nm, 
                     height = hab_spectra.raw_y.max()*.05,
                 line_width = 1, 
                 fill_color = wavelength_2_RGB_str( sentinel_array[se].center_nm, alpha=60 ) )

In [ ]:
# Plot the actual spectra
junk = spectra_w.line( hab_spectra.wavelengths,
                      hab_spectra.raw_y, 
                      legend_label = "Spectra", 
                        line_width = 3
                     )
show(spectra_w)

#| hide
### HABSpec Navigation and positioning

#| hide
#### def nearest_rgb_image( fn, debug=False):

In [ ]:
#| export
#debug_nearest_rgb_image = False
def nearest_rgb_image( 
  fn,          # Json spectra file.
  debug=False  # True to debug internals.
) -> str:      # File name of the nearest RGB photo to this json spectra.
    """
    Returns the path/filename to the RGB photo closes in time to `fn`.  `fn` is the
    filename of a Json hyperspectral file.

    Parameters:
    -----------
    fn : str
      Full path/filename of a hyperspectral Json file.

    Returns:
    --------
    str
      A string path/filename of the closes RGB photo.

    Examples:
    ---------

    References:
    -----------
    """
    if debug:
      print(f'debug_nearest_rgb_image(fn): fn={fn}')
    fn_parts = fn.split('/')   # Split path by '/'
    if debug:
      print(f'debug_nearest_rgb_image(fn): fn_parts={fn_parts}')
    rgb_p = fn_parts                          # Make a copy to build the rgb path/filename in.
    if debug:
      print(f'debug_nearest_rgb_image(fn): rgb_p={rgb_p}')    
    rgb_p[-2] = 'hab_rgb'                     # Change the subdir to point to hab_rgb
    rgb_fn = rgb_p[-1].split('-')             # Split the filename by '-' to access parts.
    rgb_fn[-2] = '*'                          # Replace the microseconds with '*' wildcard.
    rgb_fn[-1] = 'rgb.jpg'                    # Change the file tail to rgb.jpg
    rgb_p[-1] = '-'.join(rgb_fn)              # Glue the name back together
    rgb_p2 = '/'.join(rgb_p)                  # Now glue the whole path back together
    rv = glob.glob(rgb_p2)[0]                 # Return the first entry incase there are more than 1.
    if debug:
      print(f'debug_nearest_rgb_image(fn): rv={rv}') 
    return rv


In [ ]:
print(
f'    Input Json spectra file: {test_spectra_file}\n'
f'        Nearest RGB File is: {nearest_rgb_image( test_spectra_file )}'  
)

    Input Json spectra file: ../sample_data/034901/hab_spectra/2023-0718-034912-259843-spec.json
        Nearest RGB File is: ../sample_data/034901/hab_rgb/2023-0718-034912-948434-rgb.jpg


####   def read_gps_file_to_df( ifn, debug=False):

In [ ]:
#| export
#debug_read_gps_file_to_df = False
def read_gps_file_to_df( ifn, debug=False):
    '''
    Read a GPS datafile into a dataframe and convert the HH:MM:SS
    to add an SOD column.

    Parameters:
    -----------
    ifn : str
        Input GPS datafile full path name

    Returns:
    --------
    Pandas Dataframe of the GPS data.
    '''
    gps_df = pd.read_csv(ifn, sep='\s+', comment='#')
    gps_df.sort_values(by=['HMS'], inplace=True)
    # Convert the ASCII HH:MM:SS from the GPS to seconds of the day (SOD) and
    # add an 'SOD' column to the gps dataframe.
    sod_lst = []
    for t in gps_df['HMS']:
      sod_lst.append( hms2sod(t))
    gps_df['SOD'] = sod_lst

    # Compute, and add 'Course' to dataframe
    course = []
    for i in  range(len(gps_df)-1):
      course.append ( get_bearing( 
          lat1=gps_df['Lat'].iloc[i], long1=gps_df['Lon'].iloc[i], 
          lat2=gps_df['Lat'].iloc[i+1], long2=gps_df['Lon'].iloc[i+1] 
          ) 
      )
    course.append( 0.0 )      # To make the same length
    gps_df['Course'] = course
    return gps_df

Test read_gps_file_to_df() function.  Read a datafile into a Pandas dataframe.

In [ ]:
Gps_File_Name = '/mnt/f/My Drive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/2021-1208-175324-910850-mission-gps.txt'
Gps_File_Name = '../sample_data/034901/2023-0718-034805-411046-flightline-gps.txt'
gps_df = read_gps_file_to_df(Gps_File_Name, debug=False)
display(gps_df)

,Date,HMS,Lat,Lon,Elev,Speed,Nsat,Mode,Herr,Verr,Temp,SOD,Course
0,2023-07-18,03:49:01,28.209750,-82.363932,22.50,0.0,4,3,25.0,44.9,68.756,13741.0,19.402235
1,2023-07-18,03:49:02.100000,28.209751,-82.363932,22.50,0.0,4,3,25.0,44.9,68.756,13742.1,16.761456
2,2023-07-18,03:49:03.100000,28.209751,-82.363932,22.40,0.0,4,3,25.0,44.9,68.756,13743.1,30.312737
3,2023-07-18,03:49:04,28.209752,-82.363932,22.40,0.0,4,3,25.0,44.9,68.756,13744.0,60.193171
4,2023-07-18,03:49:05,28.209752,-82.363931,22.30,0.0,4,3,25.0,44.9,68.218,13745.0,-16.761454
5,2023-07-18,03:49:06.100000,28.209753,-82.363931,22.50,0.0,4,3,25.0,44.9,68.756,13746.1,46.024777
6,2023-07-18,03:49:07.100000,28.209754,-82.363930,22.40,0.0,4,3,25.0,44.9,68.756,13747.1,16.294989
7,2023-07-18,03:49:08.200000,28.209755,-82.363929,22.30,0.0,4,3,25.0,44.9,69.294,13748.2,138.464356
8,2023-07-18,03:49:09,28.209754,-82.363929,22.30,0.0,4,3,25.0,44.9,69.294,13749.0,41.535644
9,2023-07-18,03:49:10,28.209755,-82.363929,22.30,0.0,4,3,25.0,44.9,69.294,13750.0,33.467529


####   def compute_gps_positions(  gps, spec_df, debug=False):
See: https://numpy.org/doc/stable/reference/generated/numpy.interp.html

In [ ]:
#| export
#debug_compute_gps_positions = False 
def compute_gps_positions(  gps, spec_df, debug=False):
    '''
    Interpolates spectra positions from gps.

    Parameters:
    -----------
    gps : dataframe
        A dataframe of overlapping gps position data.

    spec_df : dataframe
        A dataframe of values vs time from the spectrometer.

    Returns:
    --------
    See: https://numpy.org/doc/stable/reference/generated/numpy.interp.html
    '''
    spec_df['Lat']    = np.interp(spec_df['sod'], gps['SOD'], gps['Lat'])
    spec_df['Lon']    = np.interp(spec_df['sod'], gps['SOD'], gps['Lon'])
    spec_df['Elev']   = np.interp(spec_df['sod'], gps['SOD'], gps['Elev'])
    spec_df['Course'] = np.interp(spec_df['sod'], gps['SOD'], gps['Course'])
    return spec_df

#### def get_bearing( lat1=0, lat2=0, long1=0, long2=0, debug=False):

In [ ]:
#| export
#debug_get_bearing = False
def get_bearing( lat1=0, lat2=0, long1=0, long2=0, debug=False):
    '''
    Comutes and returns the bearing between two lat/lon pairs.
    See:
        http://shorturl.at/atGHN
    '''
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

In [ ]:
# Test get_bearing()
lst = [ [30,29, -75, -75],
        [29,30, -75, -75],
        [30,30, -76, -75],
        [30,30, -75, -76] ]
print('\nTesting get_bearing()')
for t in lst:
  v = get_bearing( lat1=t[0], lat2=t[1], long1=t[2], long2=t[3] )
  print( f'  {t}, {v:6.2f}')


Testing get_bearing()
  [30, 29, -75, -75], 180.00
  [29, 30, -75, -75],   0.00
  [30, 30, -76, -75],  89.75
  [30, 30, -75, -76], -89.75


#### def get_list_of_flight_lines( p, debug=False):

In [ ]:
#| export
#debug_get_list_of_flight_lines = False
def get_list_of_flight_lines( p, debug=False):
    '''Returns a list of flightline subdirs on path p.

    Parameters:
    -----------
    p : str
        Path name.

    Returns:
    --------
    list
        A list of full pathnames to individul flightlines.
    '''
    l = glob.glob(p+'/[0-9]*[!a-z]')
    return l

Test get_list_of_flight_lines().

In [ ]:
print('\nTesting: get_list_of_flight_lines(p)')
test_path_list = get_list_of_flight_lines( mission_folder )
display(test_path_list[0:5])


Testing: get_list_of_flight_lines(p)


[]

#### def get_list_of_hab_files( p, subdir='', ext='', debug=False):

In [ ]:
#| export
#debug_get_list_of_hab_files = False
def get_list_of_hab_files( p, subdir='', ext='', debug=False):
    '''Returns a list of hab files in p/subdir with the specified file extension.  
    
    Parameters:
    p : str
    subdir : str Default = ''
    ext : str Default = ''

    Returns:
    --------
    list
        A list of  full pathnames.
    '''
    gs = p+f'/{subdir}/*.'+ext
    js = glob.glob(gs)
    return js

Test get_list_of_hab_files().

In [ ]:
#mission_folder = '/mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/'
print('\nTesting: get_list_of_hab_files()')
flight_line_list = get_list_of_flight_lines(mission_folder)
rv_get_list_of_hab_files = get_list_of_hab_files( flight_line_list[1], subdir='hab_spectra', ext='json')
display(rv_get_list_of_hab_files[0:5])

In [ ]:
print(f'{now_utc()}: HyperSpectral functions loaded.')

2023-0721 03:39:37: HyperSpectral functions loaded.


In [ ]:
rv_get_list_of_hab_files

In [ ]:
mission_folder, flight_line_list 

('../sample_data/', ['../sample_data/034901'])

In [ ]:
flight_line_list

['../sample_data/034901']

### Depreciated, to be removed.

In [ ]:
band_500_600[0][-1]

520

####   def get_fluorescence( x, y, fl_start=0, fl_stop=0, base_start=0, base_stop=0, debug=False ):

In [ ]:
#| export
#debug_get_fluorescence = False
def get_fluorescence( x, y, fl_start=0, fl_stop=0, base_start=0, base_stop=0, debug=False ):
    '''
    '''
    fl_sig = y[w_range_pixels(x, fl_start, fl_stop )].mean()         # Get the Fluor signal mean value
    by = y[w_range_pixels(x, base_start, base_start+1)].mean()
    ey = y[w_range_pixels(x, base_stop, base_stop+1)].mean()
    center_nm = (fl_stop - fl_start) / 2  + fl_start                                       # Compute center wavelength
    dydx = (ey - by)/(base_stop - base_start )
    sf = dydx * (center_nm - base_start)
    rv = fl_sig + sf
    if debug:
      print('get_fluorescence():', fl_sig, by, ey, center_nm, dydx, sf, rv)
    return rv

#### def get_fluorescence_700( x, y):

In [ ]:
#| export
def get_fluorescence_700( x, y):
    '''
    Returns the fluorescence value at 700nm.
    '''
    rv = get_fluorescence( x, y, fl_start=693, fl_stop=710, base_start=668, base_stop=740 )
    return rv

#### def get_fluorescence_683( x, y):

In [ ]:
#| export
def get_fluorescence_683( x, y):
    '''
    Returns the fluorescence value at 683nm.  683nm is Chlorophyll
    '''
    rv = get_fluorescence( x, y, fl_start=678, fl_stop=688, base_start=668, base_stop=740 )
    return rv